<a href="https://colab.research.google.com/github/NehaDas25/MLOps-Project-Pipeline/blob/main/TestModel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ML Pipelines using ZenML

*   Convert existing ML code to ML pipelines using ZenML.
*   Integrate and build models using sklearn.



In [9]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2   #Recommended by collab.

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

NumExpr defaulting to 2 threads.
⠇ Installing integrations...


{'status': 'ok', 'restart': True}

In [ ]:
NGROK_TOKEN = "2aET30xD1eZBsX7juRFmeSwtZ9z_6c7Pht9FboeMLtnkrRw7F"  ## My Grok Token

In [2]:
from zenml.environment import Environment
if Environment.in_google_colab():
  # Install and authenticate ngrok
  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


ZenML Setup

Using this tool to use and integrate advanced MLOPs to run ML pipelines.

In [3]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
ZenML repository initialized at /content.
⠼ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.

The local active stack was initialized to 'default'. This local configuration will only take effect 
when you're running ZenML from the initialized repository root, or from a subdirectory. For more 
information on repositories and configurations, please visit 
https://docs.zenml.io/user-guide/starter-guide/understand-stacks.


Experimenting ML code
*   Training a scikit-learn SVC classifier to classify the images of handwritten digits.
*   Load the data, train a model on training set and test it on testing set.



In [4]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
  """Train and Test Scikit-learn SVC classifier on digits"""
  digits = load_digits()
  data= digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(
      data, digits.target, test_size=0.2, shuffle=False
  )
  model = SVC(gamma=0.001)
  model.fit(X_train,y_train)
  test_acc = model.score(X_test, y_test)
  print(f"Test accuraccy: {test_acc}")

train_test()

Test accuraccy: 0.9583333333333334


Convert above experimentation into ML Pipeline using ZenML.
*   Data loading using importer
*   Model training using SVC trainer from SVM classifier.
*   Model Evaluation using Python evaluator.

*Using Python decorator @step for ZenML pipeline*





In [5]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:
    """Load the digits datasets as numpy arrays."""
    digits = load_digits()
    data= digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=False
    )
    return X_train, X_test, y_train, y_test

@step
def svc_trainer(X_train:np.ndarray, y_train:np.ndarray,) -> ClassifierMixin:
  """Train a sklearn SVC classifier"""
  model = SVC(gamma=0.001)
  model.fit(X_train,y_train)
  return model


@step
def evaluator(X_test:np.ndarray, y_test:np.ndarray,model: ClassifierMixin,) -> float:
  """Calculate the test accuracy of the Sklearn model"""
  test_acc = model.score(X_test, y_test)
  print(f"Test accuraccy: {test_acc}")
  return test_acc


Use ZenML pipeline decorator to connect and run the whole pipeline.

In [6]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  """ Link all the steps together in a pipelinbe"""
  X_train, X_test, y_train, y_test = importer()
  model = svc_trainer(X_train=X_train, y_train=y_train)
  evaluator(X_test=X_test, y_test=y_test, model=model)

Running zenml pipeline using run() method.

In [7]:
digits_svc_pipeline = digits_pipeline()
#digits_pipeline().run(unlisted=True)

Initiating a new run for the pipeline: digits_pipeline.
Reusing registered version: (version: 1).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Using cached version of importer.
Step importer has started.
Using cached version of svc_trainer.
Linking artifact output to model None version None implicitly.
Step svc_trainer has started.
Using cached version of evaluator.
Linking artifact output to model None version None implicitly.
Step evaluator has started.
Run digits_pipeline-2023_12_29-22_07_43_090853 has finished in 0.680s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


Visualize ZenML dashboard using zenml environment

In [ ]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok

    public_url = ngrok.connect(port)
    print(f"\x1b[31mIn Colab, use this URL instead : {public_url}!\x1b[0m")
    !zenml up --blocking --port {port}

  else:
    !zenml up --port {port}

start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-24e0cc12-b653-4057-b550-6d38ac6678f5


Opening tunnel named: http-8237-24e0cc12-b653-4057-b550-6d38ac6678f5


INFO:pyngrok.process.ngrok:t=2023-12-29T22:15:06+0000 lvl=info msg="no configuration paths supplied"


t=2023-12-29T22:15:06+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2023-12-29T22:15:06+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2023-12-29T22:18:52+0000 lvl=info msg="join connections" obj=join id=e84d50073507 l=127.0.0.1:8237 r=208.52.57.68:64779
Got token from cookie
INFO:     208.52.57.68:0 - "GET /api/v1/runs?pipeline_id=70d43328-d991-42e7-b053-bf8ee39a5058&sort_by=desc:created&logical_operator=and&page=1&size=10 HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2023-12-29T22:18:58+0000 lvl=info msg="join connections" obj=join id=192d993646d6 l=127.0.0.1:8237 r=208.52.57.68:64779


t=2023-12-29T22:18:58+0000 lvl=info msg="join connections" obj=join id=192d993646d6 l=127.0.0.1:8237 r=208.52.57.68:64779
Got token from cookie
INFO:     208.52.57.68:0 - "GET /api/v1/runs?pipeline_id=70d43328-d991-42e7-b053-bf8ee39a5058&sort_by=desc:created&logical_operator=and&page=1&size=10 HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2023-12-29T22:19:04+0000 lvl=info msg="join connections" obj=join id=55f0eb22cefe l=127.0.0.1:8237 r=208.52.57.68:64779


t=2023-12-29T22:19:04+0000 lvl=info msg="join connections" obj=join id=55f0eb22cefe l=127.0.0.1:8237 r=208.52.57.68:64779
Got token from cookie
INFO:     208.52.57.68:0 - "GET /api/v1/runs?pipeline_id=70d43328-d991-42e7-b053-bf8ee39a5058&sort_by=desc:created&logical_operator=and&page=1&size=10 HTTP/1.1" 200 OK
